In [2]:
import polars as pl

In [3]:
var_df = pl.read_excel('variables.xlsx')
var_df = (var_df
 .with_columns(
     pl.when(pl.col('fieldName').is_null()).then(pl.lit('true')).otherwise(pl.lit('false')).alias('disabled')
 )
 .with_columns(pl.col('fieldName').fill_null('null'))
 .with_columns(pl.col('bivariateField').fill_null('null'))
 .with_columns(pl.col('moransField').fill_null('null'))
 .with_columns(pl.col('yearsAvailable').fill_null('[]'))
 .with_row_index(name='variableId',offset=1)
 .select(pl.col('variableId','name','fieldName','bivariateField','moransField',
                'valueType','yearsAvailable','disabled','category'))
 
)
var_df

variableId,name,fieldName,bivariateField,moransField,valueType,yearsAvailable,disabled,category
u32,str,str,str,str,str,str,str,str
1,"""Population - Older Adult""","""percentover65""","""z_percentover65""","""sig_quad_percentover65""","""percentage""","""[2018,2019,2020,2021,2022,2023…","""false""","""Character"""
2,"""Population - Youth""","""percentunder18""","""z_percentunder18""","""sig_quad_percentunder18""","""percentage""","""[2018,2019,2020,2021,2022,2023…","""false""","""Character"""
3,"""Age or Residents""","""median_agee""","""z_median_agee""","""sig_quad_median_agee""","""numeric""","""[]""","""false""","""Character"""
4,"""Area""","""medianhouseholdincomee""","""z_medianhouseholdincomee""","""sig_quad_medianhouseholdincome…","""numeric""","""[]""","""false""","""Character"""
5,"""Population Density""","""null""","""null""","""null""","""numeric""","""[]""","""true""","""Character"""
…,…,…,…,…,…,…,…,…
83,"""Bicycle Friendliness""","""null""","""null""","""null""","""numeric""","""[]""","""true""","""Transportation"""
84,"""Proximity to Public Transporta…","""null""","""null""","""null""","""percentage""","""[]""","""true""","""Transportation"""
85,"""Sidewalk Availablity""","""null""","""null""","""null""","""percentage""","""[]""","""true""","""Transportation"""


In [4]:
v_dicts = var_df.select(pl.col('variableId','name','fieldName','bivariateField','moransField',
                               'valueType','yearsAvailable','disabled')).to_dicts()

In [5]:
with open('reg_exp_variables.json','w') as f:
    for var in v_dicts:
        print(var)
        f.write('{\n')
        for k,v in var.items():
            rec =  f"{k}: '{v}'" \
                if k in ['valueType','name','fieldName','bivariateField','moransField'] \
                            else f"{k}: {v}"
            f.write(f"{rec},\n")
        f.write('},\n')

{'variableId': 1, 'name': 'Population - Older Adult', 'fieldName': 'percentover65', 'bivariateField': 'z_percentover65', 'moransField': 'sig_quad_percentover65', 'valueType': 'percentage', 'yearsAvailable': '[2018,2019,2020,2021,2022,2023]', 'disabled': 'false'}
{'variableId': 2, 'name': 'Population - Youth', 'fieldName': 'percentunder18', 'bivariateField': 'z_percentunder18', 'moransField': 'sig_quad_percentunder18', 'valueType': 'percentage', 'yearsAvailable': '[2018,2019,2020,2021,2022,2023]', 'disabled': 'false'}
{'variableId': 3, 'name': 'Age or Residents', 'fieldName': 'median_agee', 'bivariateField': 'z_median_agee', 'moransField': 'sig_quad_median_agee', 'valueType': 'numeric', 'yearsAvailable': '[]', 'disabled': 'false'}
{'variableId': 4, 'name': 'Area', 'fieldName': 'medianhouseholdincomee', 'bivariateField': 'z_medianhouseholdincomee', 'moransField': 'sig_quad_medianhouseholdincomee', 'valueType': 'numeric', 'yearsAvailable': '[]', 'disabled': 'false'}
{'variableId': 5, 'nam

In [6]:
var_df.group_by('category').agg(pl.col('variableId')).sort(by='category').to_pandas()

,category,variableId
0,Character,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]"
1,Economy,"[13, 14, 15, 16, 17, 18, 19, 20, 21]"
2,Education,"[22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33]"
3,Engagement,"[34, 35, 36, 37]"
4,Environment,"[38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]"
5,Health,"[49, 50, 51, 52, 53, 54, 55, 56, 57]"
6,Housing,"[58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 6..."
7,Safety,"[74, 75, 76, 77, 78, 79]"
8,Transportation,"[80, 81, 82, 83, 84, 85, 86, 87, 88]"
